# <center><a href='https://notebook.basthon.fr/?from=https://raw.githubusercontent.com/mp2i-fsm/mp2i-2021/main/ds/ds7/ds7_cor.ipynb'>DS 7 corrigé : Transformée de Burrows-Wheeler et compression bzip <img src=https://framagit.org/uploads/-/system/project/avatar/55763/basthon_shadow.png width=100></a></center>

## Partie I. Compression par redondance

In [1]:
void print_array(int* t, int n) { // pour vérification
    for(int i = 0; i < n; i++)
        printf("%d ", t[i]);
}

int t[] = {0, 0, 3, 2, 3, 3, 3, 3, 3, 3, 5};
int n = 11;

error: compile_error

In [2]:
// Q1. (/1.5)
int* occurrences(int* t, int n) {
    int* r = (int*)malloc(256*sizeof(int));
    for(int i = 0; i < 256; i++)
        r[i] = 0;
    for(int i = 0; i < n; i++)
        r[t[i]]++;
    return r;
}

print_array(occurrences(t, n), 256)

error: compile_error

In [3]:
// Q2. (/1)
int min(int* t, int n) {
    int* r = occurrences(t, n);
    int m = 0;
    for(int i = 1; i < 256; i++)
        if(r[i] < r[m])
            m = i;
    return m;
}

min(t, n)

error: compile_error

In [4]:
// Q3. (/2)
int tailleCodage(int* t, int n) {
    int nprime = 1;
    for(int i = 0; i < n; i++) {
        if(i < n - 1 && t[i] == t[i + 1]) {
            while(t[i] == t[i + 1])
                i++;
            nprime += 3;
        } else
            nprime++;
    }
    return nprime;
}

tailleCodage(t, n)

error: compile_error

In [5]:
// Q4. (/3)
int* codage(int* t, int n) {
    int mu = min(t, n);
    int nprime = tailleCodage(t, n);
    int* tprime = (int*)malloc(nprime*sizeof(int));
    int k = 1;
    tprime[0] = mu;
    for(int j = 0; j < n; j++) {
        if(j < n - 1 && t[j] == t[j + 1]) {
            int i = j;
            while(t[j] == t[i])
                j++;
            j--;
            tprime[k++] = mu;
            tprime[k++] = j - i;
        }
        tprime[k++] = t[j];
    }
    return tprime;
}

int* tprime = codage(t, n);
int nprime = tailleCodage(t, n);
print_array(tprime, nprime)

error: compile_error

In [6]:
// Q4. (/4)
int* decodage(int* tprime, int nprime) {
    int mu = tprime[0];
    
    // calcul de la taille du tableau à créer
    int n = 0;
    for(int i = 1; i < nprime; i++) {
        n++;
        if(tprime[i] == mu)
            n += tprime[(++i)++];
    }
    
    int* t = (int*)malloc(n*sizeof(int));
    
    for(int i = 1, k = 0; k < n; i++) {
        if(tprime[i] == mu) {
            for(int j = 0; j <= tprime[i + 1]; j++)
                t[k++] = tprime[i + 2];
            i += 2;
        }
        else
            t[k++] = tprime[i];
    }
    return t;
}

print_array(decodage(tprime, nprime), n)

error: compile_error

## Partie II. Transformation de Burrows-Wheeler

In [7]:
let occurrences t =
  let n = Array.length t in
  let r = Array.make 256 0 in
  for i = 0 to n - 1 do
    r.(Char.code t.(i)) <- r.(Char.code t.(i)) + 1
  done;
  r

val occurrences : char array -> int array = <fun>


In [8]:
(* Q6. (/2) *)
let comparerRotations t i j =
  let n = String.length t in
  let res = ref 0 in
  for k = 0 to n - 1 do
    if !res = 0 then (
      if t.[(i + k) mod n] < t.[(j + k) mod n]
      then res := 1
      else if t.[(i + k) mod n] > t.[(j + k) mod n]
      then res := -1
    )
  done;
  !res;;

let t = "concours";;
comparerRotations t 0 3

val comparerRotations : string -> int -> int -> int = <fun>


val t : string = "concours"


In [9]:
let triRotations text = 
    let n = String.length text in
    let rot = Array.init n (fun i -> i) in
    Array.sort (fun i j -> comparerRotations text j i) rot;
    rot;;
    
let tri = triRotations t;;
tri

val triRotations : string -> int array = <fun>


val tri : int array = [|0; 3; 2; 1; 4; 6; 7; 5|]


- : int array = [|0; 3; 2; 1; 4; 6; 7; 5|]


In [10]:
(* Q7. (/2) *)
let codageBW text = (* O(nlog(n)) *)
    let n = String.length text in
    let tri = triRotations t in (* O(nlog(n)) *)
    let ans = Array.make n 'a' in (* O(n) *)
    let key = ref (-1) in
    for i = 0 to n - 1 do (* O(n) *)
        ans.(i) <- text.[(n - 1 + tri.(i)) mod n];
        if ans.(i) = text.[0] && !key = -1
        then key := i
    done;
    ans, !key;;

let code, _ = codageBW t;;

val codageBW : string -> char array * int = <fun>


val code : char array = [|'s'; 'n'; 'o'; 'c'; 'c'; 'u'; 'r'; 'o'|]


## Partie III. Transformation de Burrows-Wheeler inverse

In [11]:
(* Q9 (/2) *)
let triCarsDe tprime =
    let n = Array.length tprime in
    let triCars = Array.make n 'a' in
    let occ = occurrences tprime  in
    let k = ref 0 in
    for i = 0 to Array.length occ - 1 do
        for j = 0 to occ.(i) - 1 do
            triCars.(!k) <- Char.chr i;
            incr k
        done;
    done;
    triCars;;
    
triCarsDe code

val triCarsDe : char array -> char array = <fun>


- : char array = [|'c'; 'c'; 'n'; 'o'; 'o'; 'r'; 's'; 'u'|]
